In [1]:
import polars

from autofeat.dataset import *
from autofeat.transform import *

In [2]:
datasets = [
    KaggleDataset(id="mahmoudemadabdallah/hr-analytics-employee-attrition-and-performance"),
]

In [3]:
tables = [
    table
    for dataset in datasets
    for table in dataset.list_tables()
]

In [4]:
transform = (
    Cast()
    .then(Aggregate(by={"EmployeeID"}))
    .then(Identity(), Join(on={"EmployeeID"}))
    .then(Identity(), Combine())
)

In [5]:
results = polars.collect_all([
    table.data.select(polars.all().name.suffix(f" from {table.name}")) 
    for table in transform.apply(tables)
])

In [6]:
features = polars.concat(
    [
        df.select(polars.selectors.numeric()) 
        for df in results
        if len(df) == 1
    ],
    how="horizontal",
)

In [7]:
features

count(*) from RatingLevel.csv,max(RatingID) from RatingLevel.csv,mean(RatingID) from RatingLevel.csv,median(RatingID) from RatingLevel.csv,min(RatingID) from RatingLevel.csv,sum(RatingID) from RatingLevel.csv,std(RatingID) from RatingLevel.csv,var(RatingID) from RatingLevel.csv,count(*) from SatisfiedLevel.csv,max(SatisfactionID) from SatisfiedLevel.csv,mean(SatisfactionID) from SatisfiedLevel.csv,median(SatisfactionID) from SatisfiedLevel.csv,min(SatisfactionID) from SatisfiedLevel.csv,sum(SatisfactionID) from SatisfiedLevel.csv,std(SatisfactionID) from SatisfiedLevel.csv,var(SatisfactionID) from SatisfiedLevel.csv,count(*) from EducationLevel.csv,max(EducationLevelID) from EducationLevel.csv,mean(EducationLevelID) from EducationLevel.csv,median(EducationLevelID) from EducationLevel.csv,min(EducationLevelID) from EducationLevel.csv,sum(EducationLevelID) from EducationLevel.csv,std(EducationLevelID) from EducationLevel.csv,var(EducationLevelID) from EducationLevel.csv,count(*) + max(RatingID) from RatingLevel.csv,count(*) - max(RatingID) from RatingLevel.csv,max(RatingID) - count(*) from RatingLevel.csv,count(*) * max(RatingID) from RatingLevel.csv,count(*) / max(RatingID) from RatingLevel.csv,max(RatingID) / count(*) from RatingLevel.csv,count(*) + mean(RatingID) from RatingLevel.csv,count(*) - mean(RatingID) from RatingLevel.csv,mean(RatingID) - count(*) from RatingLevel.csv,count(*) * mean(RatingID) from RatingLevel.csv,count(*) / mean(RatingID) from RatingLevel.csv,mean(RatingID) / count(*) from RatingLevel.csv,count(*) + median(RatingID) from RatingLevel.csv,…,var(EducationLevelID) / median(EducationLevelID) from EducationLevel.csv,min(EducationLevelID) + sum(EducationLevelID) from EducationLevel.csv,min(EducationLevelID) - sum(EducationLevelID) from EducationLevel.csv,sum(EducationLevelID) - min(EducationLevelID) from EducationLevel.csv,min(EducationLevelID) * sum(EducationLevelID) from EducationLevel.csv,min(EducationLevelID) / sum(EducationLevelID) from EducationLevel.csv,sum(EducationLevelID) / min(EducationLevelID) from EducationLevel.csv,min(EducationLevelID) + std(EducationLevelID) from EducationLevel.csv,min(EducationLevelID) - std(EducationLevelID) from EducationLevel.csv,std(EducationLevelID) - min(EducationLevelID) from EducationLevel.csv,min(EducationLevelID) * std(EducationLevelID) from EducationLevel.csv,min(EducationLevelID) / std(EducationLevelID) from EducationLevel.csv,std(EducationLevelID) / min(EducationLevelID) from EducationLevel.csv,min(EducationLevelID) + var(EducationLevelID) from EducationLevel.csv,min(EducationLevelID) - var(EducationLevelID) from EducationLevel.csv,var(EducationLevelID) - min(EducationLevelID) from EducationLevel.csv,min(EducationLevelID) * var(EducationLevelID) from EducationLevel.csv,min(EducationLevelID) / var(EducationLevelID) from EducationLevel.csv,var(EducationLevelID) / min(EducationLevelID) from EducationLevel.csv,sum(EducationLevelID) + std(EducationLevelID) from EducationLevel.csv,sum(EducationLevelID) - std(EducationLevelID) from EducationLevel.csv,std(EducationLevelID) - sum(EducationLevelID) from EducationLevel.csv,sum(EducationLevelID) * std(EducationLevelID) from EducationLevel.csv,sum(EducationLevelID) / std(EducationLevelID) from EducationLevel.csv,std(EducationLevelID) / sum(EducationLevelID) from EducationLevel.csv,sum(EducationLevelID) + var(EducationLevelID) from EducationLevel.csv,sum(EducationLevelID) - var(EducationLevelID) from EducationLevel.csv,var(EducationLevelID) - sum(EducationLevelID) from EducationLevel.csv,sum(EducationLevelID) * var(EducationLevelID) from EducationLevel.csv,sum(EducationLevelID) / var(EducationLevelID) from EducationLevel.csv,var(EducationLevelID) / sum(EducationLevelID) from EducationLevel.csv,std(EducationLevelID) + var(EducationLevelID) from EducationLevel.csv,std(EducationLevelID) - var(EducationLevelID) from EducationLevel.csv,var(EducationLevelID) - std(EducationLevelID) from EducationLevel.csv,std(EducationLevelID) * v